In [0]:
import numpy as np
import pandas as pd
import matplotlib as plt
import statsmodels.api as sm
from scipy import stats
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from patsy import dmatrices

In [0]:
!pip install git+https://github.com/madrury/py-glm.git
from glm.glm import GLM
from glm.families import Gaussian, Bernoulli, Poisson, Exponential

  Cloning https://github.com/madrury/py-glm.git to /tmp/pip-req-build-wlqxk_2q
  Stored in directory: /tmp/pip-ephem-wheel-cache-3e1hy55o/wheels/ba/f8/77/41a1cc9756ab50434bbadffdf60a20742fc01bfd5ea3e056eb
Successfully built py-glm


In [0]:
linear_model = GLM(family=Gaussian())
logistic_model = GLM(family=Bernoulli())
poisson_model = GLM(family=Poisson())
exponential_model = GLM(family=Exponential())

In [0]:
clean=pd.read_csv('cleaned.csv')
clean.head()

,Unnamed: 0,cancel,year,zip.code,house.color,ni.age,len.at.res,credit,coverage.type,dwelling.type,premium,sales.channel,ni.gender,ni.marital.status,n.adults,n.children,tenure,claim.ind,state
0,0,0,2013,98004.0,blue,37.0,18.000000,high,C,Tenant,950.507336,Broker,F,0.0,2.0,0.0,15.0,0.0,WA
1,1,0,2013,85017.0,white,40.0,17.000000,medium,A,Condo,909.346046,Broker,F,1.0,5.0,0.0,15.0,0.0,AZ
2,2,0,2013,80021.0,white,45.0,14.000000,high,A,House,897.084502,Phone,M,0.0,1.0,0.0,14.0,0.0,CO
3,3,0,2013,85002.0,red,45.0,24.366136,high,A,House,979.039007,Phone,M,0.0,5.0,0.0,22.0,1.0,AZ
4,4,0,2013,50001.0,yellow,36.0,16.000000,high,C,House,932.379027,Broker,M,1.0,2.0,4.0,4.0,0.0,IA


In [0]:
clean.columns = ['index', 'cancel','year', 'zip','color','age','len','credit','coverage','dwelling','premium','channel','gender','marital','adults','children','tenure','claim','state']

In [0]:
clean_color = pd.get_dummies(clean['color'])
clean_credit = pd.get_dummies(clean['credit'])
clean_coverage = pd.get_dummies(clean['coverage'])
clean_dwelling = pd.get_dummies(clean['dwelling'])
clean_channel = pd.get_dummies(clean['channel'])
clean_gender = pd.get_dummies(clean['gender'])
clean_state = pd.get_dummies(clean['state'])
clean = clean.join(clean_color)
clean = clean.join(clean_credit)
clean = clean.join(clean_coverage)
clean = clean.join(clean_dwelling)
clean = clean.join(clean_channel)
clean = clean.join(clean_gender)
clean = clean.join(clean_state)
clean.head()

,Unnamed: 0,cancel,year,zip,color,age,len,credit,coverage,dwelling,...,Online,Phone,F,M,AZ,CO,IA,PA,VA,WA
0,0,0,2013,98004.0,blue,37.0,18.000000,high,C,Tenant,...,0,0,1,0,0,0,0,0,0,1
1,1,0,2013,85017.0,white,40.0,17.000000,medium,A,Condo,...,0,0,1,0,1,0,0,0,0,0
2,2,0,2013,80021.0,white,45.0,14.000000,high,A,House,...,0,1,0,1,0,1,0,0,0,0
3,3,0,2013,85002.0,red,45.0,24.366136,high,A,House,...,0,1,0,1,1,0,0,0,0,0
4,4,0,2013,50001.0,yellow,36.0,16.000000,high,C,House,...,0,0,0,1,0,0,1,0,0,0


In [0]:
clean2013=clean.loc[clean['year'] == 2013]
clean2014=clean.loc[clean['year'] == 2014]
clean2015=clean.loc[clean['year'] == 2015]
clean2016=clean.loc[clean['year'] == 2016]

In [0]:
list(clean)

['Unnamed: 0',
 'cancel',
 'year',
 'zip',
 'color',
 'age',
 'len',
 'credit',
 'coverage',
 'dwelling',
 'premium',
 'channel',
 'gender',
 'marital',
 'adults',
 'children',
 'tenure',
 'claim',
 'state',
 'blue',
 'red',
 'white',
 'yellow',
 'high',
 'low',
 'medium',
 'A',
 'B',
 'C',
 'Condo',
 'House',
 'Tenant',
 'Broker',
 'Online',
 'Phone',
 'F',
 'M',
 'AZ',
 'CO',
 'IA',
 'PA',
 'VA',
 'WA']

In [0]:
logistic_model=sm.formula.glm(formula="cancel ~ age + len + premium + marital + adults + children + tenure + claim",family=sm.families.Binomial(), data=clean).fit()
logistic_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 cancel   No. Observations:              1025272
Model:                            GLM   Df Residuals:                  1025263
Model Family:                Binomial   Df Model:                            8
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:            -5.5252e+05
Date:                Mon, 18 Mar 2019   Deviance:                   1.1050e+06
Time:                        00:38:46   Pearson chi2:                 1.03e+06
No. Iterations:                     5                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6152      0.030     20.307      0.000       0.556       0.675
age           -0.0294      0.000   -105.485      0.000      -0.030      -0.029
len           -0.0200      0.001    -35.245      0.000      -0.021      -0.019
premium       -0.0007   3.01e-05    -24.650      0.000      -0.001      -0.001
marital       -0.3370      0.006    -59.703      0.000      -0.348      -0.326
adults         0.0712      0.002     39.005      0.000       0.068       0.075
children       0.1123      0.001    108.549      0.000       0.110       0.114
tenure         0.0234      0.001     46.160      0.000       0.022       0.024
claim          0.3565      0.006     63.765      0.000       0.346       0.367
==============================================================================
"""

In [81]:
logistic_model=sm.formula.glm(formula="cancel ~ age + len + premium + marital + adults + children + tenure + claim + blue + red + white + yellow + low + high + medium + A + B + C + Condo + House + Tenant + Broker + Online + Phone + F + M + AZ + CO + IA + PA + VA + WA",family=sm.families.Binomial(), data=clean).fit()
logistic_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 cancel   No. Observations:              1025272
Model:                            GLM   Df Residuals:                  1025246
Model Family:                Binomial   Df Model:                           25
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:            -5.0362e+05
Date:                Mon, 18 Mar 2019   Deviance:                   1.0072e+06
Time:                        01:41:22   Pearson chi2:                 1.02e+06
No. Iterations:                     5                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3053      0.010     30.951      0.000       0.286       0.325
age           -0.0269      0.000    -90.204      0.000      -0.027      -0.026
len           -0.0222      0.001    -37.065      0.000      -0.023      -0.021
premium       -0.0008   3.18e-05    -26.225      0.000      -0.001      -0.001
marital       -0.3757      0.006    -63.000      0.000      -0.387      -0.364
adults         0.0800      0.002     41.463      0.000       0.076       0.084
children       0.1260      0.001    114.842      0.000       0.124       0.128
tenure         0.0303      0.001     56.248      0.000       0.029       0.031
claim          0.3975      0.006     67.147      0.000       0.386       0.409
blue           0.0779      0.005     15.683      0.000       0.068       0.088
red            0.0714      0.005     13.976      0.000       0.061       0.081
white          0.0780      0.005     16.792      0.000       0.069       0.087
yellow         0.0780      0.007     11.100      0.000       0.064       0.092
low            0.8223      0.005    156.354      0.000       0.812       0.833
high          -0.5712      0.005   -123.552      0.000      -0.580      -0.562
medium         0.0542      0.005     10.685      0.000       0.044       0.064
A              0.0415      0.005      8.591      0.000       0.032       0.051
B              0.0945      0.005     18.460      0.000       0.084       0.105
C              0.1693      0.005     36.353      0.000       0.160       0.178
Condo          0.0942      0.005     18.520      0.000       0.084       0.104
House          0.0662      0.005     13.675      0.000       0.057       0.076
Tenant         0.1449      0.005     27.613      0.000       0.135       0.155
Broker        -0.4068      0.005    -82.155      0.000      -0.416      -0.397
Online         0.2405      0.007     36.874      0.000       0.228       0.253
Phone          0.4715      0.005     92.125      0.000       0.462       0.482
F              0.1513      0.006     27.395      0.000       0.140       0.162
M              0.1540      0.006     27.945      0.000       0.143       0.165
AZ             0.0613      0.005     11.903      0.000       0.051       0.071
CO            -0.2929      0.006    -50.598      0.000      -0.304      -0.282
IA            -0.0491      0.005     -9.882      0.000      -0.059      -0.039
PA            -0.5356      0.007    -80.664      0.000      -0.549      -0.523
VA             1.0552      0.009    123.828      0.000       1.039       1.072
WA             0.0664      0.007      9.913      0.000       0.053       0.080
==============================================================================
"""

In [79]:
logistic_model=sm.formula.glm(formula="cancel ~ age + len + premium + marital + adults + children + tenure + claim + red + white + yellow + high + medium + B + C + House + Tenant + Online + Phone + M + CO + IA + PA + VA + WA",family=sm.families.Binomial(), data=clean).fit()
logistic_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 cancel   No. Observations:              1025272
Model:                            GLM   Df Residuals:                  1025246
Model Family:                Binomial   Df Model:                           25
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:            -5.0362e+05
Date:                Mon, 18 Mar 2019   Deviance:                   1.0072e+06
Time:                        01:40:30   Pearson chi2:                 1.02e+06
No. Iterations:                     5                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1469      0.033     34.316      0.000       1.081       1.212
age           -0.0269      0.000    -90.204      0.000      -0.027      -0.026
len           -0.0222      0.001    -37.065      0.000      -0.023      -0.021
premium       -0.0008   3.18e-05    -26.225      0.000      -0.001      -0.001
marital       -0.3757      0.006    -63.000      0.000      -0.387      -0.364
adults         0.0800      0.002     41.463      0.000       0.076       0.084
children       0.1260      0.001    114.842      0.000       0.124       0.128
tenure         0.0303      0.001     56.248      0.000       0.029       0.031
claim          0.3975      0.006     67.147      0.000       0.386       0.409
red           -0.0065      0.007     -0.955      0.340      -0.020       0.007
white          0.0001      0.006      0.023      0.982      -0.012       0.012
yellow      7.402e-05      0.009      0.008      0.994      -0.018       0.019
high          -1.3935      0.006   -222.027      0.000      -1.406      -1.381
medium        -0.7681      0.007   -107.257      0.000      -0.782      -0.754
B              0.0530      0.007      7.994      0.000       0.040       0.066
C              0.1278      0.006     22.384      0.000       0.117       0.139
House         -0.0279      0.006     -4.570      0.000      -0.040      -0.016
Tenant         0.0507      0.007      6.921      0.000       0.036       0.065
Online         0.6473      0.009     74.188      0.000       0.630       0.664
Phone          0.8783      0.005    166.933      0.000       0.868       0.889
M              0.0027      0.005      0.549      0.583      -0.007       0.012
CO            -0.3542      0.008    -45.712      0.000      -0.369      -0.339
IA            -0.1104      0.007    -16.105      0.000      -0.124      -0.097
PA            -0.5969      0.009    -68.688      0.000      -0.614      -0.580
VA             0.9939      0.011     92.242      0.000       0.973       1.015
WA             0.0051      0.009      0.586      0.558      -0.012       0.022
==============================================================================
"""

In [83]:
y_true = clean['cancel']
y_scores = logistic_model.predict()
roc_auc_score(y_true, y_scores)

0.7263643193606434

In [0]:
model = sm.formula.glm("cancel ~ house.color+ni.age+len.at.res+credit+coverage.type+dwelling.type+premium+sales.channel+ni.gender+ni.marital.status+n.adults+n.children+tenure+claim.ind+state",
                       family=sm.families.Binomial(), data=clean).fit()
print(model.summary())

In [0]:

fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)